In [1]:
import argparse
import pandas as pd
import re

In [2]:
scflp_path = "scflp_all.csv"
lgda_path  = "lgda_results_avg_nomask_20260115_204139.csv"

In [3]:


def pick_col(df: pd.DataFrame, candidates: list[str], required: bool = True) -> str | None:
    for c in candidates:
        if c in df.columns:
            return c
    if required:
        raise KeyError(f"None of columns {candidates} found. Available: {list(df.columns)}")
    return None


def load_scflp(path: str, beta: float | None = None, status_ok: int | None = 1) -> pd.DataFrame:
    df = pd.read_csv(path)

    # optional filters
    if beta is not None and "beta" in df.columns:
        df = df[df["beta"] == beta]
    if status_ok is not None and "status" in df.columns:
        df = df[df["status"] == status_ok]

    gcols = ["I", "J", "p", "r"]
    obj_col = pick_col(df, ["obj", "theta", "share", "share_rate"])
    time_col = pick_col(df, ["time_sec", "time", "runtime_sec", "seconds"])

    out = (
        df.groupby(gcols, as_index=False)
          .agg(mip_share=(obj_col, "mean"),
               mip_time=(time_col, "mean"))
    )
    return out


def load_lgda(path: str) -> pd.DataFrame:
    df = pd.read_csv(path)
    gcols = ["I", "J", "p", "r"]

    share_col = pick_col(df, ["obj_binary_avg", "lgda_share", "share_avg", "obj_avg", "obj"])
    time_col  = pick_col(df, ["time_avg_sec", "lgda_time", "time_sec", "time"])
    full_col  = pick_col(df, ["obj_ex_avg", "full_share", "exhaustive_share", "obj_full_avg"], required=False)

    keep = gcols + [share_col, time_col] + ([full_col] if full_col else [])
    out = df[keep].copy()
    out = out.rename(columns={share_col: "lgda_share", time_col: "lgda_time"})
    if full_col:
        out = out.rename(columns={full_col: "full_share"})
    return out


def render_table(df: pd.DataFrame,
                 caption: str,
                 label: str,
                 nd_share: int = 3,
                 nd_time: int = 2,
                 order_pr=((2, 2), (3, 2), (2, 3))) -> str:
    pr_rank = {pr: i for i, pr in enumerate(order_pr)}

    df = df.copy()
    df["pr_rank"] = df.apply(lambda r: pr_rank.get((int(r["p"]), int(r["r"])), 999), axis=1)
    df = df.sort_values(["I", "J", "pr_rank"])

    def fshare(x): return f"{float(x):.{nd_share}f}"
    def ftime(x):  return f"{float(x):.{nd_time}f}"

    lines = []
    has_full = "full_share" in df.columns
    for _, r in df.iterrows():
        inst = f"{int(r['I'])}-{int(r['J'])}-{int(r['p'])}-{int(r['r'])}"
        full = fshare(r["full_share"]) if has_full and pd.notna(r.get("full_share")) else "-"
        lines.append(
            f"    {inst:<12} &{full}& {fshare(r['mip_share'])} & {ftime(r['mip_time'])}  & "
            f"{fshare(r['lgda_share'])} & {ftime(r['lgda_time'])} \\\\"
        )

    header = r"""\begin{table}[tb]
  \centering
  \caption{CAPTION}
  \label{LABEL}
  \begin{tabular}{c|c|cc|cc}
    \hline
    \begin{tabular}{c}
      事例 \\
      $(|I|-|J|-p-r)$
    \end{tabular}
    & 全探索
    & \multicolumn{2}{c|}{MIP（文献値）}
    & \multicolumn{2}{c}{Mutation（提案手法）} \\
    \cline{2-6}
    & シェア率 & シェア率 & 計算時間 (s) & シェア率 & 計算時間 (s) \\
    \hline
"""
    footer = r"""    \hline
  \end{tabular}
\end{table}
"""
    return header.replace("CAPTION", caption).replace("LABEL", label) + "\n".join(lines) + "\n" + footer


In [4]:
scflp_df = load_scflp(scflp_path, status_ok=None)  # ← これで status フィルタ無効化
lgda_df  = load_lgda(lgda_path)

df = pd.merge(scflp_df, lgda_df, on=["I","J","p","r"], how="inner")

In [5]:
full_log = """
試行 1/1 完了: obj=0.5022911346678027
試行 1/1 完了: obj=0.6058935726556893
試行 1/1 完了: obj=0.4088499332116071
試行 1/1 完了: obj=0.5025174272892741
試行 1/1 完了: obj=0.6043303940239402
試行 1/1 完了: obj=0.4048399837252744
試行 1/1 完了: obj=0.5019137430606436
試行 1/1 完了: obj=0.6041792620928732
試行 1/1 完了: obj=0.40288564828521645
試行 1/1 完了: obj=0.5016426166090029
試行 1/1 完了: obj=0.6010602312818973
試行 1/1 完了: obj=0.4012494701123786
試行 1/1 完了: obj=0.5009353979700619
試行 1/1 完了: obj=0.6026450483446272
試行 1/1 完了: obj=0.4022068325142957
試行 1/1 完了: obj=0.5009353979700619
試行 1/1 完了: obj=0.6026450483446272
試行 1/1 完了: obj=0.4022068325142957
試行 1/1 完了: obj=0.5009353979700619
試行 1/1 完了: obj=0.6026450483446272
試行 1/1 完了: obj=0.4022068325142957
"""

full_vals = [float(x) for x in re.findall(r"obj=([0-9.]+)", full_log)]

# render_table と同じ順番で並べてから、上から順に full_share を入れる
order_pr = ((2, 2), (3, 2), (2, 3))
pr_rank = {pr: i for i, pr in enumerate(order_pr)}

df2 = df.copy()
keys = list(zip(df2["p"].astype(int), df2["r"].astype(int)))
df2["pr_rank"] = [pr_rank.get(k, 999) for k in keys]

df2 = df2.sort_values(["I", "J", "pr_rank"]).reset_index(drop=True)

if len(full_vals) != len(df2):
    raise ValueError(f"full_vals の個数({len(full_vals)})と表の行数({len(df2)})が一致しません。")

df2["full_share"] = full_vals
df = df2.drop(columns=["pr_rank"])

In [6]:
tex = render_table(df,
                   caption=r"~\citet{qi:or:2024} と提案手法のシェア率および計算時間",
                   label="tb:mulrow")
print(tex)

\begin{table}[tb]
  \centering
  \caption{~\citet{qi:or:2024} と提案手法のシェア率および計算時間}
  \label{tb:mulrow}
  \begin{tabular}{c|c|cc|cc}
    \hline
    \begin{tabular}{c}
      事例 \\
      $(|I|-|J|-p-r)$
    \end{tabular}
    & 全探索
    & \multicolumn{2}{c|}{MIP（文献値）}
    & \multicolumn{2}{c}{Mutation（提案手法）} \\
    \cline{2-6}
    & シェア率 & シェア率 & 計算時間 (s) & シェア率 & 計算時間 (s) \\
    \hline
    20-20-2-2    &0.502& 0.525 & 3.08  & 0.503 & 110.18 \\
    20-20-3-2    &0.606& 0.629 & 8.20  & 0.602 & 101.12 \\
    20-20-2-3    &0.409& 0.447 & 2.86  & 0.404 & 94.10 \\
    40-40-2-2    &0.503& 0.537 & 10.17  & 0.503 & 95.88 \\
    40-40-3-2    &0.604& 0.627 & 57.88  & 0.602 & 96.97 \\
    40-40-2-3    &0.405& 0.442 & 6.69  & 0.402 & 95.48 \\
    60-60-2-2    &0.502& 0.537 & 20.78  & 0.501 & 95.09 \\
    60-60-3-2    &0.604& 0.620 & 182.34  & 0.601 & 90.59 \\
    60-60-2-3    &0.403& 0.436 & 28.28  & 0.401 & 90.75 \\
    80-80-2-2    &0.502& 0.528 & 72.04  & 0.501 & 91.11 \\
    80-80-3-2    &0.601& 0.6